---
## Installs

In [1]:
!pip install pytube
!pip install imagehash

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 819kB 6.8MB/s 
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295206 sha256=83343e2a1ab7b5444263c3636a479d84cdf8e063b639ac10d5f658c01609b88a
  Stored in directory: /root/.cache/pip/wheels/32/3c/c7/b0653122fc94b27b4c60d2260896bb4759e56396c6720d0a6c
Successfully built imagehash


## Import Libraries

In [8]:
from pytube import YouTube
import os
import cv2
import time
import re
import numpy as np
import imagehash
from PIL import Image

---
## Parameters (link, path)

In [3]:
# pdf'e dönüştürülmesi istenen video linki
link = "https://www.youtube.com/watch?v=fHLhBnmwUM0"

# Video adı için kullanılan değişken
filename = "deneme"

---
## Download the model and save screenshots

In [4]:
def downloadYouTube(videourl, filename):
  """
  Youtube'dan video indirmeyi sağlar.
  videourl : indirilecek olan video'nun linki {str}
  filename : indirdikten sonra isimlendirilmesi için kullanılacak ad {str}
  """

  yt = YouTube(videourl)
  yt = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
  yt.download(filename = filename)
  return yt, yt.title

yt, title = downloadYouTube(link,filename)

In [5]:
img_count = 0
counter = 0
success = True

# Screenshot'ların kaydedileceği path:
path = "/content/screenshots/"

# video'nun olduğu path'i alıp videoyu okuyorum:
video_path = f"/content/{filename}.mp4"
vidcap = cv2.VideoCapture(video_path)

# Eğer screenshot klasörü yoksa aç veya devam et diyorum:
if not os.path.exists(path):
    os.makedirs(path)

# Screenshot'ları oluşturmak için while'a aldım.
while success:
  success,image = vidcap.read()
  counter+=1

  # Frame frame ss alırken bunların sırasını da korumak benim için önemli.
  # Nedeni ise pdf oluştururken de bu sırayı alacağım için. Eğer karışık olursa
  # pdf de karışık ve sırasız gelen frame'lerden oluşur.
  if counter%10 == 0 :
    try : 
      if img_count < int(10) :
        dst =path + "000" + str(img_count) + ".jpg"
        cv2.imwrite(dst, image)
        img_count +=1

      if img_count >= int(10) and img_count < int(100) :
        dst =path + "00" + str(img_count) + ".jpg"
        cv2.imwrite(dst, image)
        img_count +=1

      if img_count>= int(100) and img_count< int(1000) :
        dst =path + "0" + str(img_count) + ".jpg"
        cv2.imwrite(dst, image)
        img_count +=1
      
      if img_count>= int(1000) :
        dst =path + str(img_count) + ".jpg"
        cv2.imwrite(dst, image)
        img_count +=1
    except : #Video bittiğinde hata geliyor o yüzden pass diyerek bitirdim.
      pass

/content/deneme.mp4


---
## Compare the differences in sequential images

In [9]:
#Kaç adet screenshot'ım olduğunu öğreniyorum:
num_of_images = len(os.listdir(path))

# Düzgün numaralandırarak isimlendirdiğim screenshot'ları sıralıyorum:
images = sorted(os.listdir(path))

# Belirlenen farktan fazla değişiklik olan frame'in ekleneceği liste
alınacaklar = []

# kaç bit değişiklik olacağı bilgisini veriyorum. Bir nevi threshold gibi düşünülebilir.
cutoff = 5

for x, _ in enumerate(images):
  #Eğer toplam ss sayımdan bir eksiğine geldiysem döngüden çık.
  if x == num_of_images-1:
    break
  else:
    #Peşi sıra gelen iki imgenin piksel olarak farkını alıp benzerlik yakalamaya çalışıyorum:
    imge1 = imagehash.average_hash(Image.open(path + images[x]))
    imge2 = imagehash.average_hash(Image.open(path + images[x+1]))
    # Benzerlik belirlediğim değerden büyükse bu imgeyi alınacaklar listesine ekle diyorum:
    if imge1 - imge2 > cutoff:
      alınacaklar.append(images[x])

In [10]:
# Alınacaklar listesi haricindekileri aldım. Ekrana bastırmaması için de boş bir değişkene atadım.
_ = [os.remove(path + str(image)) for image in images if image not in alınacaklar]

---
## Image to pdf in order

In [16]:
imagelist = []

# Yine ekran görüntülerini sıraladım.
# Bunu yapmamın nedeni pdf olacağı zaman sıranın korunması.
images = sorted(os.listdir(path))

# Elimdeki ss'leri okuyup RGB'ye çevirdim ve listeye attım.
for x, image in enumerate(images): 
  if x>0:
    image1 = Image.open(path + image, mode="r")
    im1 = image1.convert('RGB')
    imagelist.append(im1)

# İlk ekran görüntüsünü ayrıca alıyorum. Bunu yapmamın sebebi pdf oluştururken
# bunun üzerinden fonksiyonu çağıracağım.
image1 = Image.open(path + images[0], mode="r")
im1 = image1.convert('RGB')

# pdf kaydedilirken kullanılacak isim videonun ismi olması için title verdim
saved_name = f"{title}.pdf"

#ilk 4-5 frame intro olduğu için onları almayacağım:
#Sizin için farklılık gösterebilir.
imagelist = imagelist[4:]

# İlk sayfa üzerinden save fonksiyonunu çağırdım.
# Kaydedilecek ismi verdikten sonra tüm resimleri sırasıyla append'laması
# için imagelist'imi verdim.
im1.save(saved_name,save_all=True, append_images=imagelist)
print(f"{title} saved with pdf format.")

StatQuest: Boxplots, Clearly Explained saved with pdf format.


---
## İŞİN BİTİNCE ÇALIŞTIR!! (Ekran Görüntülerinin Hepsini Siler)

In [13]:
""" 
Videodan oluşturulan tüm ekran görüntüleri silmek için kullanılır.
Bellekte yer tutmaması için yazdım ancak content'te çalışıyorsanız
bunu çalıştırmak zorunda değilsiniz oturum kapanınca content'in 
içindekiler zaten otomatik silinecektir.
"""
#[os.remove(path + str(file)) for file in os.listdir(path) if file.endswith('.jpg')]